# Introduction to cirq 2: variational methods

In this tutorial we will apply the quantum alternating operator ansatz (QAOA) to find the ground state of a random-bond Ising model.

1. VQE construction
    1. VQE ansatz construction
        1. Initial state preparation
        1. Circuit generators
    1. Cost function construction
1. VQE simulation and optimization
    1. Simulating a single VQE instance
    1. Symbolic parameters
    1. Grid search with sweeps
    1. Choosing a classical optimization strategy
    
(This tutorial is adapted from the Cirq tutorial at https://cirq.readthedocs.io/en/stable/tutorial.html)

# VQE construction
## VQE ansatz construction
Recall that a VQE consists of
1. A parameterized quantum circuit
2. A starting state
3. A cost function and a method for measuring it
4. A classical optimization outer loop

The QAOA is a popular variational circuit that consists of repeating two layers of operators - one defined by the starting state, one by the target state.
In particular, in each layer we evolve by $e^{iH_at}$, where $H_0$ is chosen so that our starting state is an eigenstate, and $H_1$ is a Hamiltonian that defines the problem we wish to solve.
This is a popular strategy for many applications; optimization and quantum simulation to name a few.

In this tutorial we will focus on constructing the QAOA ansatz for a native target problem - the random bond Ising model. This is defined by a Hamiltonian
$$
H_1=\sum_{\langle i,j\rangle}t_{i,j}Z_iZ_j,
$$
where $\langle i,j\rangle$ denotes nearest neighbours on some lattice, and $t_{i,j}=\pm 1$.

We note that this is a terrible problem to try to solve on a quantum device, as the ground state is known to be QMA-hard to prepare. But it still makes a good example problem :).

For a starting state, let us take the uniform superposition across all ground states - $\otimes_i|+\rangle$. The corresponding Hamiltonian is then
$$
H_0=\sum_{i}X_i.
$$

Let us implement the algorithm on a square lattice of size $n_x\times n_y$. If we set $n_x=n_y=3$ this should be quite quick to simulate.

### Initial state preparation
We first need to construct a circuit to prepare our initial state

In [1]:
import cirq
nx = 3
ny = 3
def init_state_circuit(nx, ny):
     yield [cirq.H(cirq.GridQubit(i,j)) for i in range(nx) for j in range(ny)]

circuit = cirq.Circuit(init_state_circuit(nx, ny))
print(circuit)

(0, 0): ───H───

(0, 1): ───H───

(0, 2): ───H───

(1, 0): ───H───

(1, 1): ───H───

(1, 2): ───H───

(2, 0): ───H───

(2, 1): ───H───

(2, 2): ───H───


### Circuit construction

Now we need to make the two blocks of our circuit. Let us first set our random Hamiltonian.

In [2]:
import numpy as np
rng = np.random.RandomState(seed=42)

# H1_right[i,j] defines the link between qubit i,j and qubit i+1,j
H1_right = np.array([[rng.choice([+1, -1]) for j in range(ny)] for i in range(nx-1)])
# H1_down[i,j] defines the link between qubit i,j and qubit i,j+1
H1_down = np.array([[rng.choice([+1, -1]) for j in range(ny-1)] for i in range(nx)])

# To check that we have the right dimensions and deal with the standard
# x, y
print(H1_right.T)
print(H1_down.T)

[[ 1  1]
 [-1  1]
 [ 1 -1]]
[[ 1  1  1]
 [ 1 -1  1]]


Defining our circuit, we need to be careful; if we arrange the gates for H1 at random it will not be constant depth. The trick here is to use a clockwork format - pick one qubit, and do gates to the right, then down, then left, then up around this qubit (and all gates facing in the same direction that fit with this one).

**Problem 1:** Write code to implement evolution by $e^{iH_1\theta}$ (I've started it off for you).

In [3]:
def H0_block(alpha, nx, ny):
    xgate = cirq.XPowGate(exponent=alpha)
    yield [xgate(cirq.GridQubit(i,j)) for i in range(nx) for j in range(ny)]

def H1_block(H1_right, H1_down, beta, nx, ny):
    zzgate = cirq.ZZPowGate(exponent=beta)
    # Right gates
    yield [zzgate(cirq.GridQubit(i, j), cirq.GridQubit(i+1, j))**H1_right[i,j]
           for i in range(0,nx-1,2) for j in range(ny)]

    # Down gates
    yield [zzgate(cirq.GridQubit(i, j), cirq.GridQubit(i, j+1))**H1_down[i,j]
           for i in range(nx) for j in range(0,ny-1,2)]
    # Left gates
    yield [zzgate(cirq.GridQubit(i, j), cirq.GridQubit(i+1, j))**H1_right[i,j]
           for i in range(1,nx-1,2) for j in range(ny)]
    # Up gates
    yield [zzgate(cirq.GridQubit(i, j), cirq.GridQubit(i, j+1))**H1_down[i,j]
           for i in range(nx) for j in range(1,ny-1,2)]
    
circuit = cirq.Circuit(H1_block(H1_right, H1_down, 0.5, nx, ny))
# Check that this is equal to 4!
print('Number of blocks in circuit = {}'.format(len(circuit)))

#Print the circuit to check
print(circuit)

Number of blocks in circuit = 4
           ┌───────────────────┐             ┌────────────┐
(0, 0): ────ZZ─────────────────────ZZ───────────────────────────────────
            │                      │
(0, 1): ────┼─────ZZ───────────────ZZ^0.5─────ZZ────────────────────────
            │     │                           │
(0, 2): ────┼─────┼──────ZZ───────────────────ZZ^0.5────────────────────
            │     │      │
(1, 0): ────ZZ^0.5┼──────┼─────────ZZ─────────ZZ────────────────────────
                  │      │         │          │
(1, 1): ──────────ZZ^-0.5┼─────────ZZ^0.5─────┼─────ZZ────────ZZ────────
                         │                    │     │         │
(1, 2): ─────────────────ZZ^0.5────ZZ─────────┼─────┼─────────ZZ^-0.5───
                                   │          │     │
(2, 0): ────ZZ─────────────────────┼──────────ZZ^0.5┼───────────────────
            │                      │                │
(2, 1): ────ZZ^0.5─────────────────┼────────────────ZZ^0.5────ZZ─

We now need to handle measurement of the system - or rather how the measurement will be stored classically. In order for ease of access, we can read out all the qubits in a single measurement step, but we need to be worried about qubit order here --- or at least, that we know what order we have chosen for future access.

In [4]:
def msmt_circuit(nx, ny):
    msmt_order = [cirq.GridQubit(i,j) for i in range(nx) for j in range(ny)]
    yield cirq.measure(*msmt_order, key='Zmeasurements')
    
# Just to show how to go between the two values    
test = [cirq.GridQubit(i,j) for i in range(nx) for j in range(ny)]
print(test)
array_reset = np.array([[test[i*ny+j] for j in range(ny)] for i in range(nx)])
print(array_reset)
print(array_reset[1,2])

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]
[[cirq.GridQubit(0, 0) cirq.GridQubit(0, 1) cirq.GridQubit(0, 2)]
 [cirq.GridQubit(1, 0) cirq.GridQubit(1, 1) cirq.GridQubit(1, 2)]
 [cirq.GridQubit(2, 0) cirq.GridQubit(2, 1) cirq.GridQubit(2, 2)]]
(1, 2)


The full circuit now consists of initialization, $p$ repetitions of the two parts of the ansatz, and finally measurement. Note that we should do the evolution by $H_1$ before the evolution by $H_0$, and that each repetition should be controlled by a different parameter.

**Problem 2:** Write a function to construct the full QAOA circuit

In [5]:
def QAOA_circuit(H1_right, H1_down, p, alpha_vec, beta_vec, nx, ny):
    '''
    Args:
        H1_right: Matrix defining the horizontal couplings in the random bond Ising model
            H1_right[i,j] couples qubits (i,j) and (i+1,j)
        H1_down: Matrix defining the vertical couplings in the random bond Ising model
            H1_down[i,j] couples qubits (i,j) and (i,j+1)
        p: number of layers in the ansatz
        alpha_vec(list of p floats): list of alpha values for each QAOA layer
        beta_vec(list of p floats): list of beta values for each QAOA layer
        nx, ny(integers): dimensions of model
    '''

    yield init_state_circuit(nx, ny)
    for index in range(p):
        yield H1_block(H1_right, H1_down, beta_vec[index], nx, ny)
        yield H0_block(alpha_vec[index], nx, ny)
    yield msmt_circuit(nx, ny)

### Cost function construction
We now need to write code to calculate our cost function on the system. In this case we will use $\langle H_1\rangle$. As we are measuring our wavefunction in the $Z$ basis, we may calculate the cost function immediately for each instance of the system and then just average over this.

**Problem 3:** write a function to calculate the energy of the random bond ising model given a set of measurements

In [6]:
def energy_func(H1_right, H1_down, nx, ny):
    def energy(measurements):
        # Rearrange to match the qubit array + change from 0 and 1 to 1 and -1
        msmt_array = 1 - 2 * np.array([[measurements[i*ny+j] for j in range(ny)]
                                        for i in range(nx)]).astype(int)

        energy_row = sum([H1_right[i][j] * msmt_array[i,j] * msmt_array[i+1,j]
                          for i in range(nx-1) for j in range(ny)])
        energy_column = sum([H1_down[i][j] * msmt_array[i,j] * msmt_array[i,j+1]
                             for i in range(nx) for j in range(ny-1)])
        return energy_row + energy_column
        
    return energy

Our cost function is then the average over the results of multiple measurements.

In [7]:
def cost_func(result, H1_right, H1_down, nx, ny):
    energy_hist = result.histogram(key='Zmeasurements',
                                   fold_func=energy_func(H1_right, H1_down, nx, ny))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions

# VQE simulation and optimization
## Simulating a single VQE instance
Before we go any further, let's use what we learnt last week to simulate an instance of our VQE using some fixed parameters and measuring the output.

**Problem 4:** Simulate the above VQE for $p=1$, and $\beta,\alpha=0.25$. If you average over 100000 instances you should get an energy somewhere between $4.6-4.7$.

In [8]:
simulator = cirq.Simulator()
beta_vec = [0.25]
alpha_vec = [0.25]
p = 1
circuit = cirq.Circuit(QAOA_circuit(H1_right, H1_down, p, alpha_vec, beta_vec, nx, ny))
results = simulator.run(circuit, repetitions=100000)
print(cost_func(results, H1_right, H1_down, nx, ny))

4.61668


### Symbolic parameters

We want the parameters $\alpha$ and $\beta$ to be optimized during the VQE. This is most easily done by passing around sympy.Symbol values, which work well with cirq.

In [9]:
import sympy
beta_vec = [sympy.Symbol('beta{}'.format(j)) for j in range(p)]
alpha_vec = [sympy.Symbol('alpha{}'.format(j)) for j in range(p)]
circuit = cirq.Circuit(QAOA_circuit(H1_right, H1_down, p, alpha_vec, beta_vec, nx, ny))
print(circuit)

               ┌───────────────────────────┐                 ┌────────────────┐
(0, 0): ───H────ZZ─────────────────────────────ZZ─────────────X^alpha0─────────────────────────────────────M('Zmeasurements')───
                │                              │                                                           │
(0, 1): ───H────┼───────ZZ─────────────────────ZZ^beta0───────ZZ──────────────────X^alpha0─────────────────M────────────────────
                │       │                                     │                                            │
(0, 2): ───H────┼───────┼──────────ZZ─────────────────────────ZZ^beta0────────────X^alpha0─────────────────M────────────────────
                │       │          │                                                                       │
(1, 0): ───H────ZZ^beta0┼──────────┼───────────ZZ─────────────ZZ──────────────────X^alpha0─────────────────M────────────────────
                        │          │           │              │              

As you can see, some of the circuits now have gates that are parameterized. These can be resolved in cirq, most usefully by a 'sweep' over the parameters. This may be done via the cirq.Linspace object, which is a set of instructions to resolve a certain parameter that may be chained together, and the simulator.run_sweep function, which can interpret this. Let's try to run this with our simple $p=1$ algorithm.

In [11]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha0', start=0, stop=1, length=sweep_size)
        * cirq.Linspace(key='beta0', start=0, stop=1, length=sweep_size))
results = simulator.run_sweep(circuit, params=sweep, repetitions=10000)

for result in results:
    print(result.params.param_dict, cost_func(result, H1_right, H1_down, nx, ny))

OrderedDict([('alpha0', 0.0), ('beta0', 0.0)]) 0.011
OrderedDict([('alpha0', 0.0), ('beta0', 0.1111111111111111)]) -0.0124
OrderedDict([('alpha0', 0.0), ('beta0', 0.2222222222222222)]) -0.0014
OrderedDict([('alpha0', 0.0), ('beta0', 0.3333333333333333)]) 0.0212
OrderedDict([('alpha0', 0.0), ('beta0', 0.4444444444444444)]) -0.0298
OrderedDict([('alpha0', 0.0), ('beta0', 0.5555555555555556)]) -0.0386
OrderedDict([('alpha0', 0.0), ('beta0', 0.6666666666666666)]) 0.0252
OrderedDict([('alpha0', 0.0), ('beta0', 0.7777777777777778)]) 0.0234
OrderedDict([('alpha0', 0.0), ('beta0', 0.8888888888888888)]) 0.0026
OrderedDict([('alpha0', 0.0), ('beta0', 1.0)]) -0.0694
OrderedDict([('alpha0', 0.1111111111111111), ('beta0', 0.0)]) -0.023
OrderedDict([('alpha0', 0.1111111111111111), ('beta0', 0.1111111111111111)]) 2.3914
OrderedDict([('alpha0', 0.1111111111111111), ('beta0', 0.2222222222222222)]) 3.1184
OrderedDict([('alpha0', 0.1111111111111111), ('beta0', 0.3333333333333333)]) 2.1034
OrderedDict([('

Now that you have the ingredients to run the VQE, all you need to do is minimize and find the result.

**Problem 5:** Find the minimum parameters and corresponding energy in the $p=1$ VQE. Use whatever method you want.

In [12]:
import time

In [13]:
p=1
beta_vec = [sympy.Symbol('beta{}'.format(j)) for j in range(p)]
alpha_vec = [sympy.Symbol('alpha{}'.format(j)) for j in range(p)]
circuit = cirq.Circuit(QAOA_circuit(H1_right, H1_down, p, alpha_vec, beta_vec, nx, ny))

In [14]:
sweep_size = 10
sweep = (cirq.Linspace(key='alpha0', start=0, stop=1, length=sweep_size)
        * cirq.Linspace(key='beta0', start=0, stop=1, length=sweep_size))
start_time_1 = time.time()
results = simulator.run_sweep(circuit, params=sweep, repetitions=1000)

min_sweep_params = []
min_sweep_energy = 0.0
for result in results:
    if cost_func(result, H1_right, H1_down, nx, ny) < min_sweep_energy:
        min_sweep_energy =  cost_func(result, H1_right, H1_down, nx, ny)
        min_sweep_params.append(result.params.param_dict)
end_time_1 = time.time()

In [15]:
print('For p=1 time: {}s'.format((end_time_1-start_time_1)))

For p=1 time: 3.9695281982421875s


In [16]:
print('Sweeped minimized energy: {}'.format(min_sweep_energy))

Sweeped minimized energy: -4.752


**Problem 6:** Find the minimum parameters and corresponding energy in the QAOA with $p=2$. Compare the time taken to find the minimum and the result to the $p=1$ result.

In [74]:
p=2
beta_vec = [sympy.Symbol('beta{}'.format(j)) for j in range(p)]
alpha_vec = [sympy.Symbol('alpha{}'.format(j)) for j in range(p)]
circuit = cirq.Circuit(QAOA_circuit(H1_right, H1_down, p, alpha_vec, beta_vec, nx, ny))

In [ ]:
sweep_size = 5
sweep = (cirq.Linspace(key='alpha0', start=0, stop=1, length=sweep_size) 
         * cirq.Linspace(key='alpha1', start=0, stop=1, length=sweep_size)
         * cirq.Linspace(key='beta0', start=0, stop=1, length=sweep_size)
          * cirq.Linspace(key='beta1', start=0, stop=1, length=sweep_size))
start_time_2 = time.time()
results = simulator.run_sweep(circuit, params=sweep, repetitions=1000)

min_sweep_params_2 = []
min_sweep_energy_2 = 0.0
for result in results:
    if cost_func(result, H1_right, H1_down, nx, ny) < min_sweep_energy:
        min_sweep_energy_2 =  cost_func(result, H1_right, H1_down, nx, ny)
        min_sweep_params_2.append(result.params.param_dict)
end_time_2 = time.time()

In [ ]:
print('For p=2 time: {}s'.format((end_time_2-start_time_2)))

In [ ]:
print('Sweeped minimized energy: {}'.format(min_sweep_energy_2))